# Balanced BS Tree: AVL Tree

## Agenda

1. Motives
2. "Balanced" binary trees
3. Essential mechanic: rotation
4. Out-of-balance scenarios & rotation recipes
5. Generalized AVL rebalancing (insertion)
6. Rebalancing on removal

## 1. Motives

In [ ]:
class BSTree:
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
    def __init__(self):
        self.size = 0
        self.root = None
    
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return BSTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1
        
    def __contains__(self, val):
        def contains_rec(node):
            if not node:
                return False
            elif val < node.val:
                return contains_rec(node.left)
            elif val > node.val:
                return contains_rec(node.right)
            else:
                return True
        return contains_rec(self.root)
    
    def __len__(self):
        return self.size
    
    def __delitem__(self, val):
        assert(val in self)
        def delitem_rec(node):
            if val < node.val:
                node.left = delitem_rec(node.left)
                return node
            elif val > node.val:
                node.right = delitem_rec(node.right)
                return node
            else:
                if not node.left and not node.right:
                    return None
                elif node.left and not node.right:
                    return node.left
                elif node.right and not node.left:
                    return node.right
                else:
                    to_del = node.left
                    if not to_del.right:
                        node.left = to_del.left
                    else:
                        par = to_del
                        to_del = par.right
                        while to_del.right:
                            par = par.right
                            to_del = to_del.right
                        par.right = to_del.left
                    node.val = to_del.val
                    return node
                        
        self.root = delitem_rec(self.root)
        self.size -= 1
        
    def pprint(self, width=64):
        """Attempts to pretty-print this tree's contents."""
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=n.val, width=width//2**level)
        print(repr_str)
    
    def height(self):
        """Returns the height of the longest branch of the tree."""
        def height_rec(t):
            if not t:
                return 0
            else:
                return max(1+height_rec(t.left), 1+height_rec(t.right))
        return height_rec(self.root)

In [ ]:
t = BSTree()
for x in range(6):
    t.add(x)
t.pprint()

In [ ]:
import sys
sys.setrecursionlimit(100)

t = BSTree()
for x in range(100):
    t.add(x)

## 2. "Balanced" binary trees

There are different metrics we can use to determine whether a binary tree is balanced --- for example, we could consider the *number* of nodes on either side of a given node, the *height* of the subtrees on either side of a given node, or even the specific *configuration* of nodes in subtrees on either side of a given node (perhaps we only want to permit full or complete trees).

For our purposes, we are going to define a balanced binary tree based on height. Specifically, an AVL tree (a particular type of balanced binary tree), is one where the *AVL property* holds for every node in the tree. The AVL property states that the heights of the left and right subtrees of a given node can differ by *no more than one*.

An AVL tree is also *self-balancing*. I.e., whenever the AVL property may be violated by an insertion/deletion, it must be immediately fixed --- the AVL property must always hold for all nodes in the tree after completing an insertion/deletion operation. 

How do we do this?

## 3. Essential mechanic: rotation

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            pass
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
t = AVLTree()
for x in range(6, 0, -1):
    t.add(x)
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.left.rotate_right()
t.pprint()

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
                                    
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            # detect and fix imbalance in `node` before returning
            return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
val = 50
t = AVLTree()

In [ ]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val -= 1
t.pprint()

In [ ]:
val = 0
t = AVLTree()

In [ ]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val += 1
t.pprint()

## 4. "Out-of-balance" scenarios & rotation recipes

In [ ]:
# "left-left" scenario
t = BSTree()
for x in [3, 2, 1]:
    t.add(x)
t.pprint()

In [ ]:
# "left-right" scenario
t = BSTree()
for x in [3, 1, 2]:
    t.add(x)
t.pprint()

In [ ]:
# "right-right" scenario
t = BSTree()
for x in [1, 2, 3]:
    t.add(x)
t.pprint()

In [ ]:
# "right-left" scenario
t = BSTree()
for x in [1, 3, 2]:
    t.add(x)
t.pprint()

## 5. Generalized AVL rebalancing (insertion)

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
            
        def rotate_left(self):
            pass
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
    
    @staticmethod
    def rebalance(t):
        if AVLTree.Node.height(t.left) > AVLTree.Node.height(t.right):
            if AVLTree.Node.height(t.left.left) >= AVLTree.Node.height(t.left.right):
                # left-left
                print('left-left imbalance detected')
                # fix?
            else:
                # left-right
                print('left-right imbalance detected')
                # fix?
        else:
            # right branch imbalance tests needed
            pass
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if False: # detect imbalance
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
t = AVLTree()
for x in [10, 5, 1]:
    t.add(x)
t.pprint()

In [ ]:
# broken!
t = AVLTree()
for x in [10, 5, 1, 2, 3]:
    t.add(x)
t.pprint()

## 5. Rebalancing on removal

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
        
        def rotate_left(self):
            pass
            
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
    
    @staticmethod
    def rebalance(t):
        if AVLTree.Node.height(t.left) > AVLTree.Node.height(t.right):
            if AVLTree.Node.height(t.left.left) >= AVLTree.Node.height(t.left.right):
                # left-left
                print('left-left imbalance detected')
                t.rotate_right()
            else:
                # left-right
                print('left-right imbalance detected')
                t.left.rotate_left()
                t.rotate_right()
        else:
            pass
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if abs(AVLTree.Node.height(node.left) - AVLTree.Node.height(node.right)) >= 2:
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1
        
    def __delitem__(self, val):
        assert(val in self)
        def delitem_rec(node):
            if val < node.val:
                node.left = delitem_rec(node.left)
            elif val > node.val:
                node.right = delitem_rec(node.right)
            else:
                if not node.left and not node.right:
                    return None
                elif node.left and not node.right:
                    return node.left
                elif node.right and not node.left:
                    return node.right
                else:
                    to_del = node.left
                    if not to_del.right:
                        node.left = to_del.left
                    else:
                        par = to_del
                        to_del = par.right
                        while to_del.right:
                            par = par.right
                            to_del = to_del.right
                        par.right = to_del.left
                    node.val = to_del.val

            # detect and fix imbalance
            return node
                        
        self.root = delitem_rec(self.root)
        self.size -= 1

In [ ]:
t = AVLTree()
for x in [10, 5, 15, 2]:
    t.add(x)
t.pprint()

In [ ]:
del t[15]
t.pprint()

In [ ]:
t = AVLTree()
for x in range(31, 0, -1):
    t.add(x)
t.pprint()

In [ ]:
del t[15]
del t[14]
t.pprint()

In [ ]:
del t[16]
t.pprint()